In [1]:
%matplotlib qt
import os
import pathlib
from time import time

import matplotlib.pyplot as plt
from memory_profiler import memory_usage

from fmm import Fmm
from fmm.kernel import laplace_p2p_serial

# Plotting parameters
plt.rc('font', family='serif', serif='Times')
plt.rc('text', usetex=True)
plt.rc('xtick', labelsize=8)
plt.rc('ytick', labelsize=8)
plt.rc('axes', labelsize=8)

# Dimensions for column plots
width = 3.487
height = width / 1.618

HERE = pathlib.Path(os.getcwd())
FIGURE_SAVEPATH = os.path.abspath(HERE.parent.parent / 'article/figures')

# The Impact of Choice of Compression Parameter $K$

## Generate Data

Uncomment to generate data

In [2]:
# ! fmm generate-test-data -c K_10 && fmm compute-operators -c K_10
# ! fmm generate-test-data -c K_20 && fmm compute-operators -c K_20
# ! fmm generate-test-data -c K_30 && fmm compute-operators -c K_30
# ! fmm generate-test-data -c K_40 && fmm compute-operators -c K_40
# ! rm K_50.hdf5 && fmm generate-test-data -c K_50 && fmm compute-operators -c K_50

# Memory

Test peak memory usage as a function of compression parameters

In [4]:
# Compression parameters tested
Kvec = [10, 20, 30, 40, 50]

# Load experiments
evec = [Fmm(f'K_{K}') for K in Kvec]

In [5]:
evec[0].run()

In [7]:
memvec = []

for e in evec:
    memvec.append(max(memory_usage(e.run)))

memvec = np.array(memvec)

In [8]:
memvec

array([662.87109375, 667.69921875, 671.921875  , 676.03515625,
       685.1796875 ])

## Accuracy

How does compression parameter effect accuracy? We expect $p$ digits of accuracy from theory, where $p$ is the order of the multipole/local expansions

In [3]:
# Compression parameters tested
Kvec = [10, 20, 30, 40, 50]

nruns = 7
# Container for timings
tvec = [[] for _ in range(nruns)]

# Evaluate FMM for each experiment
for i in range(nruns):
    evec = [Fmm(f'K_{K}') for K in Kvec]
    for e in evec:
        start = time()
        e.run()
        tvec[i].append(time()-start)

KeyboardInterrupt: 

In [33]:
tvec = np.array(tvec)

In [36]:
tvec.mean(axis=0)

array([ 6.88867296,  6.89062313,  6.92779234,  6.9516683 , 15.43500471])

In [37]:
tvec.std(axis=0)

array([0.1148165 , 0.0887785 , 0.10632887, 0.07703105, 0.54625184])

In [23]:
# Results of direct computation for each experiment
d = laplace_p2p_serial(evec[0].sources, evec[0].targets, evec[0].source_densities)

In [24]:
relerrvec = [abs(d-e.target_potentials[:,0])/d for e in evec]

In [25]:
relerrmean = [np.mean(err) for err in relerrvec]

In [26]:
relerrmean

[0.00015179637, 2.2398553e-05, 1.152469e-05, 3.6725994e-06, 2.4273586e-06]

In [27]:
fig, ax = plt.subplots()
fig.subplots_adjust(left=.15, bottom=.16, right=.99, top=.97)

ax.semilogy(Kvec, relerrmean, '--o', ms=3, lw=0.4, c='0.25')
ax.set_ylabel('Mean Relative Error ($\epsilon_{\ rel}$)')
ax.set_xlabel('Compression Rank ($K$)')
ax.set_xticks(Kvec)

fig.set_size_inches(width, height)
fp = FIGURE_SAVEPATH  + '/compression_accuracy.pdf'
plt.savefig(fp)
plt.show()

## Runtimes

How does compression parameter effect runtime?

In [28]:
fig, ax = plt.subplots()
fig.subplots_adjust(left=.15, bottom=.16, right=.99, top=.97)

ax.plot(Kvec, tvec, '--o', ms=3, lw=0.4, c='0.25')
ax.set_ylabel('FMM Runtime ($s$)')
ax.set_xlabel('Compression Rank ($K$)')
ax.set_xticks(Kvec)

fig.set_size_inches(width, height)
fp = FIGURE_SAVEPATH  + '/compression_runtime.pdf'
plt.savefig(fp)
plt.show()